Adaptado de ["Transferência de aprendizado e ajuste fino"](https://www.tensorflow.org/tutorials/images/transfer_learning?hl=pt-br) dos [tutoriais do Tensorflow](https://www.tensorflow.org/tutorials/).

# _Transfer learning_ e _fine-tuning_

<a href="https://colab.research.google.com/github/fabiobento/edge-ml/blob/main/computer-vision/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Neste tutorial, você aprenderá a classificar imagens de cães e gatos usando o transfer learning de uma rede pré-treinada.

Um modelo pré-treinado é uma rede salva que foi previamente treinada em um grande conjunto de dados, normalmente em uma tarefa de classificação de imagens em grande escala. Você pode usar o modelo pré-treinado como está ou usar o transfer learning para personalizar esse modelo para uma determinada tarefa.

A intuição por trás do transfer learning para classificação de imagens é que, se um modelo for treinado em um conjunto de dados grande e geral o suficiente, esse modelo servirá efetivamente como um modelo genérico do mundo visual. Assim, você pode aproveitar esses mapas de recursos aprendidos sem precisar começar do zero, treinando um modelo grande em um conjunto de dados grande.

Neste notebook, você tentará duas maneiras de personalizar um modelo pré-treinado:

1. Extração de recursos: Use as representações aprendidas por uma rede anterior para extrair recursos significativos de novas amostras. Basta adicionar um novo classificador, que será treinado do zero, sobre o modelo pré-treinado para que você possa reutilizar os mapas de recursos aprendidos anteriormente para o conjunto de dados.

 Não é necessário (re)treinar o modelo inteiro. A rede convolucional básica já contém recursos que são genericamente úteis para classificar imagens. No entanto, a parte final de classificação do modelo pré-treinado é específica para a tarefa de classificação original e, posteriormente, específica para o conjunto de classes no qual o modelo foi treinado.

1. _Fine-tuning_: Descongelar algumas das camadas superiores de uma base de modelo congelada e treinar conjuntamente as camadas do classificador recém-adicionadas e as últimas camadas do modelo de base. Isso nos permite fazer o “ajuste fino” das representações de recursos de ordem superior no modelo de base para torná-las mais relevantes para a tarefa específica.

Você seguirá o fluxo de trabalho geral de aprendizado de máquina.

1. Examinar e entender os dados
1. Crie um pipeline de entrada, neste caso usando o Keras ImageDataGenerator
1. Compor o modelo
   * Carregar o modelo de base pré-treinado (e pesos pré-treinados)
   * Empilhar as camadas de classificação na parte superior
1. Treinar o modelo
1. Avaliar o modelo


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

## Preprocessamento dos dados

### Download dos dados

Neste tutorial, você usará um conjunto de dados que contém vários milhares de imagens de cães e gatos.

Faça download e extraia um arquivo zip contendo as imagens e, em seguida, crie um `tf.data.Dataset` para treinamento e validação usando o utilitário `tf.keras.utils.image_dataset_from_directory`.

Você pode saber mais sobre o carregamento de imagens neste [tutorial](https://www.tensorflow.org/tutorials/load_data/images).

In [ ]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

BATCH_SIZE = 32
IMG_SIZE = (160, 160)

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

In [ ]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)

Mostre as primeiras nove imagens e rótulos do conjunto de treinamento:

In [ ]:
class_names = train_dataset.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

Como o conjunto de dados original não contém um conjunto de teste, você criará um.

Para fazer isso, determine quantos lotes de dados estão disponíveis no conjunto de validação usando `tf.data.experimental.cardinality` e, em seguida, mova 20% deles para um conjunto de teste.

In [5]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

In [ ]:
print('Número de lotes de validação: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Número de lotes de teste: %d' % tf.data.experimental.cardinality(test_dataset))

### Configurar o conjunto de dados para desempenho

Use a pré-busca com buffer(_buffered prefetching_) para carregar imagens do disco sem que a E/S se torne bloqueada. Para saber mais sobre esse método, consulte o guia [data performance](https://www.tensorflow.org/guide/data_performance).

In [7]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

### Use _data augmentation_

When you don't have a large image dataset, it's a good practice to artificially introduce sample diversity by applying random, yet realistic, transformations to the training images, such as rotation and horizontal flipping. This helps expose the model to different aspects of the training data and reduce [overfitting](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit). You can learn more about data augmentation in this [tutorial](https://www.tensorflow.org/tutorials/images/data_augmentation).

Quando não se tem um grande conjunto de dados de imagens, é uma boa prática introduzir artificialmente a diversidade de amostras aplicando transformações aleatórias, porém realistas, às imagens de treinamento, como rotação e inversão horizontal.

Isso ajuda a expor o modelo a diferentes aspectos dos dados de treinamento e a reduzir o [overfitting](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit).

Você pode saber mais sobre o aumento de dados neste [tutorial](https://www.tensorflow.org/tutorials/images/data_augmentation).

In [8]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

Observação: essas camadas ficam ativas somente durante o treinamento, quando você chama `Model.fit`. Elas ficam inativas quando o modelo é usado no modo de inferência em `Model.evaluate`, `Model.predict` ou `Model.call`.

Vamos aplicar repetidamente essas camadas à mesma imagem e ver o resultado.

In [ ]:
for image, _ in train_dataset.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')

### Redimensionar valores de pixel

Em breve, você fará o download do `tf.keras.applications.MobileNetV2` para usá-lo como seu modelo básico.

Esse modelo espera valores de pixel em `[-1, 1]`, mas, nesse momento, os valores de pixel em suas imagens estão em `[0, 255]`. Para redimensioná-los, use o método de pré-processamento incluído no modelo.

In [10]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

Observação: como alternativa, você pode redimensionar os valores de pixel de `[0, 255]` para `[-1, 1]` usando `tf.keras.layers.Rescaling`.

In [11]:
rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)

Observação: se estiver usando outros `tf.keras.applications`, verifique o documento da API para determinar se eles esperam pixels em `[-1, 1]` ou `[0, 1]`, ou use a função `preprocess_input` incluída.

## Crie o modelo básico a partir de convnets pré-treinadas
Você criará o modelo básico a partir do modelo **MobileNet V2** desenvolvido no Google.
- Ele é pré-treinado no conjunto de dados ImageNet, um grande conjunto de dados que consiste em 1,4 milhão de imagens e 1.000 classes.
- O ImageNet é um conjunto de dados de treinamento de pesquisa com uma ampla variedade de categorias, como `jackfruit` e `syringe`.
- Essa base de conhecimento nos ajudará a classificar gatos e cachorros em nosso conjunto de dados específico.

Primeiro, você precisa escolher a camada do MobileNet V2 que usará para extração de recursos.
- A última camada de classificação (no “topo”, já que a maioria dos diagramas de modelos de aprendizado de máquina vai de baixo para cima) não é muito útil.
- Em vez disso, você seguirá a prática comum de depender da última camada antes da operação de _flattening_.
- Essa camada é chamada de “camada de gargalo”.
- Os recursos da camada de gargalo mantêm mais generalidade em comparação com a camada final/superior.

Primeiro, instancie um modelo MobileNet V2 pré-carregado com pesos treinados no ImageNet.
- Ao especificar o argumento **include_top=False**, você carrega uma rede que não inclui as camadas de classificação na parte superior, o que é ideal para a extração de recursos.

In [12]:
# Criar o modelo básico a partir do modelo pré-treinado MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

Esse extrator de recursos converte cada imagem `160x160x3` em um bloco de recursos `5x5x1280`. Vejamos o que ele faz em um exemplo de lote de imagens:

In [ ]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

## Extração de recursos
Nesta etapa, você congelará a base convolucional criada na etapa anterior e a usará como um extrator de recursos. Além disso, você adiciona um classificador sobre ele e treina o classificador de nível superior.

### Congelar a base convolucional

É importante congelar a base convolucional antes de compilar e treinar o modelo.

O congelamento (definindo layer.trainable = False) impede que os pesos em uma determinada camada sejam atualizados durante o treinamento.

O MobileNet V2 tem muitas camadas, portanto, definir o sinalizador `trainable` do modelo inteiro como False congelará todas elas.

In [14]:
base_model.trainable = False

### Observação importante sobre as camadas de BatchNormalization

Muitos modelos contêm camadas `tf.keras.layers.BatchNormalization`. Essa camada é um caso especial e devem ser tomadas precauções no contexto do fine tuning, conforme mostrado mais adiante neste tutorial.

Quando você define `layer.trainable = False`, a camada `BatchNormalization` será executada no modo de inferência e não atualizará suas estatísticas de média e variância.

Quando você descongela um modelo que contém camadas de BatchNormalization para fazer o fine tuning, deve manter as camadas de BatchNormalization no modo de inferência passando `training = False` ao chamar o modelo básico. Caso contrário, as atualizações aplicadas aos pesos não treináveis destruirão o que o modelo aprendeu.

Para obter mais detalhes, consulte o [Transfer learning guide](https://www.tensorflow.org/guide/keras/transfer_learning).

In [ ]:
# Vamos dar uma olhada na arquitetura do modelo básico
base_model.summary()

### Adicionar um cabeçalho de classificação

Para gerar previsões a partir do bloco de recursos, faça a média dos valores espaciais `5x5`, usando uma camada `tf.keras.layers.GlobalAveragePooling2D` para converter os recursos em um único vetor de 1280 elementos por imagem.

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

Aplique uma camada `tf.keras.layers.Dense` para converter esses recursos em uma única previsão por imagem.

Você não precisa de uma função de ativação aqui porque essa previsão será tratada como um `logit` ou um valor de previsão bruto.

Números positivos predizem a classe 1, números negativos predizem a classe 0.

In [ ]:
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

Crie um modelo encadeando as camadas de aumento de dados, redimensionamento, `base_model` e extrator de recursos usando a [Keras Functional API](https://www.tensorflow.org/guide/keras/functional).

Conforme mencionado anteriormente, use `training=False`, pois nosso modelo contém uma camada `BatchNormalization`.

In [18]:
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.summary()

Os mais de 8 milhões de parâmetros no MobileNet estão congelados, mas há 1,2 mil parâmetros _treináveis_ na camada Dense.

Eles são divididos entre dois objetos `tf.Variable`, os pesos e as tendências.

In [ ]:
len(model.trainable_variables)

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

### Compilar o modelo

Compile o modelo antes de treiná-lo. Como há duas classes e uma saída sigmoide, use o `BinaryAccuracy`.

In [22]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5, name='accuracy')])

### Treinar o modelo

Após o treinamento de 10 épocas, você deverá ver ~96% de acurácia no conjunto de validação.


In [ ]:
initial_epochs = 10

loss0, accuracy0 = model.evaluate(validation_dataset)

In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [ ]:
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)

### Curvas de aprendizado

Vamos dar uma olhada nas curvas de aprendizado da acurácia/perda de treinamento e validação ao usar o modelo básico MobileNetV2 como um extrator de recursos fixo.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Acurácia de Treino')
plt.plot(val_acc, label='Acurácia de Validação')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Acurácias de Treino e Validação')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Perda de Treino')
plt.plot(val_loss, label='Perda de Validação')
plt.legend(loc='upper right')
plt.ylabel('Entropia Cruzada')
plt.ylim([0,1.0])
plt.title('Perdas de Treino e Validação')
plt.xlabel('epoch')
plt.show()

Observação: Se você estiver se perguntando por que as métricas de validação são claramente melhores do que as métricas de treinamento, o principal fator é que camadas como `tf.keras.layers.BatchNormalization` e `tf.keras.layers.Dropout` afetam a acurácia durante o treinamento. Elas são desativadas ao calcular a perda de validação.

Em menor escala, isso também ocorre porque as métricas de treinamento relatam a média de uma época, enquanto as métricas de validação são avaliadas após a época, de modo que as métricas de validação veem um modelo que treinou um pouco mais.

## Fine tuning

No experimento de extração de recursos, você estava treinando apenas algumas camadas sobre um modelo básico do MobileNetV2.
* Os pesos da rede pré-treinada **não** foram atualizados durante o treinamento.

Uma maneira de aumentar ainda mais o desempenho é treinar (ou “ajustar”) os pesos das camadas superiores do modelo pré-treinado juntamente com o treinamento do classificador que você adicionou.
* O processo de treinamento forçará os pesos a serem ajustados de mapas de recursos genéricos para recursos associados especificamente ao conjunto de dados.


Observação: isso só deve ser tentado depois que você tiver treinado o classificador de nível superior com o modelo pré-treinado definido como não treinável.
* Se você adicionar um classificador inicializado aleatoriamente em cima de um modelo pré-treinado e tentar treinar todas as camadas em conjunto, a magnitude das atualizações de gradiente será muito grande (devido aos pesos aleatórios do classificador) e o modelo pré-treinado esquecerá o que aprendeu.

Além disso, você deve tentar fazer o Fine tuning de um pequeno número de camadas superiores em vez de todo o modelo MobileNet.
* Na maioria das redes convolucionais, quanto mais alta for uma camada, mais especializada ela será.
* As primeiras camadas aprendem recursos muito simples e genéricos que se aplicam a quase todos os tipos de imagens.
* À medida que você sobe de nível, os recursos são cada vez mais específicos para o conjunto de dados no qual o modelo foi treinado.
* O objetivo do Fine tuning é adaptar esses recursos especializados para que funcionem com o novo conjunto de dados, em vez de substituir o aprendizado genérico.

### Descongelar as camadas superiores do modelo


Tudo o que você precisa fazer é descongelar o `base_model` e definir as camadas inferiores como não treináveis.

Em seguida, você deve recompilar o modelo (necessário para que essas alterações tenham efeito) e retomar o treinamento.

In [27]:
base_model.trainable = True

In [ ]:
# Vamos dar uma olhada para ver quantas camadas existem no modelo básico
print("Número de camadas no modelo básico: ", len(base_model.layers))

# Fine tuning a partir dessa camada
fine_tune_at = 100

# Congelar todas as camadas antes da camada `fine_tune_at
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

### Compilar o modelo

Como você está treinando um modelo muito maior e deseja readaptar os pesos pré-treinados, é importante usar uma taxa de aprendizado menor nesse estágio. Caso contrário, seu modelo poderá se ajustar muito rapidamente.

In [29]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
              metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5, name='accuracy')])

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

### Continuar o treinamento do modelo

Se você treinou para a convergência mais cedo, essa etapa aumentará sua acurácia em alguns pontos percentuais.

In [ ]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=len(history.epoch),
                         validation_data=validation_dataset)

Vamos dar uma olhada nas curvas de aprendizado da acurácia/perda de treinamento e validação ao fazer o fine tuningdas últimas camadas do modelo básico do MobileNetV2 e treinar o classificador sobre ele.
* A perda de validação é muito maior do que a perda de treinamento, portanto, pode haver um overfitting.

O overfitting também pode etar ocorrendo pois o novo conjunto de treinamento é relativamente pequeno e semelhante aos conjuntos de dados originais do MobileNetV2.


Após o fine tuning, o modelo quase atinge 98% de acurácia no conjunto de validação.

In [33]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Acurácia de Treino')
plt.plot(val_acc, label='Acurácia de Validação')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Início do fine tuning')
plt.legend(loc='lower right')
plt.title('Acurácia de Validação e Treino')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Perda de Treino')
plt.plot(val_loss, label='Perda de Validação')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Início do fine tuning')
plt.legend(loc='upper right')
plt.title('Perda de Treino e Validação')
plt.xlabel('epoch')
plt.show()

### Avaliação e previsão

Por fim, você pode verificar o desempenho do modelo em novos dados usando o conjunto de testes.

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

E agora você está pronto para usar esse modelo para prever se o seu animal de estimação é um gato ou um cachorro.

In [ ]:
# Recuperar um lote de imagens do conjunto de teste
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch).flatten()
predictions = tf.where(predictions < 0.5, 0, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n', label_batch)

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(class_names[predictions[i]])
  plt.axis("off")

## Resumo

* Uso de um modelo pré-treinado para extração de recursos**:  Ao trabalhar com um conjunto de dados pequeno, é uma prática comum aproveitar os recursos aprendidos por um modelo treinado em um conjunto de dados maior no mesmo domínio. Isso é feito instanciando o modelo pré-treinado e adicionando um classificador totalmente conectado por cima. O modelo pré-treinado é “congelado” e somente os pesos do classificador são atualizados durante o treinamento.
Nesse caso, a base convolucional extraiu todos os recursos associados a cada imagem e você acabou de treinar um classificador que determina a classe da imagem com base nesse conjunto de recursos extraídos.

* **Fine tuning de um modelo pré-treinado**: Para melhorar ainda mais o desempenho, é possível redirecionar as camadas de nível superior dos modelos pré-treinados para o novo conjunto de dados por meio do fine tuning.
Nesse caso, você ajustou os pesos de modo que o modelo aprendesse recursos de alto nível específicos do conjunto de dados. Essa técnica geralmente é recomendada quando o conjunto de dados de treinamento é grande e muito semelhante ao conjunto de dados original no qual o modelo pré-treinado foi treinado.

Para saber mais, visite o [Transfer learning guide](https://www.tensorflow.org/guide/keras/transfer_learning).
